In [2]:
import os
from mistralai import Mistral
from dotenv import load_dotenv
import pandas as pd
load_dotenv()
import time
from pymongo import MongoClient
import json
import asyncio


In [3]:
api_key = os.environ["mistral_key"]
gen_ai_key = os.environ["gen_ai_key"]
agent_id = os.environ["agent_id"]
db_url = os.environ["DB_URL"]
model = "open-mistral-nemo"
client = Mistral(api_key=api_key)
mongo_client = MongoClient(db_url)
db  = mongo_client.reddit
post_db = db['post_new']
comms_db = db['comment_db']
final_res = db['final_data']

In [5]:
rec = post_db.find()
rec = list(rec)
print(len(rec)) 

10001


In [6]:
df = pd.DataFrame(rec)

In [7]:
df = df.drop_duplicates(subset=['title'])

In [15]:
df = df[df['num_comments'] > 0]
len(df)

829

In [16]:
len(df)

829

In [20]:
comments_db = db['comment_db']
comms = comments_db.find()
comms = list(comms)
print(len(comms))
df2 = pd.DataFrame(comms)
df2 = df2.drop_duplicates(subset=['parent_id'])
len(df2)
comm__post_ids = df2['parent_id'].tolist()
comm__post_ids = [i.split('_')[1] for i in comm__post_ids]
print(comm__post_ids)


remaining_post_ids = list(set(df['id'].tolist()) - set(comm__post_ids))
print(len(remaining_post_ids))

7117
['1h9gtbi', '1h9e0an', '1h9cua9', '1h9achm', '1h9ac56', '1h980yx', '1h96sie', '1h96sbl', '1h96d8c', '1h965rt', '1h94vta', '1h94egx', '1h9347e', '1h92zk6', '1h91q7g', '1h913zy', '1h90nxb', '1h8wpup', '1h8wajl', '1h8u5d3', '1h8tnov', '1h8tgei', '1h8ss1k', '1h8r9fp', '1h8qq7b', '1h8pt5k', '1h8prw9', '1h8mm84', '1h8kbbb', '1h8jv31', '1h8iw8c', '1h8ivih', '1h8iqv1', '1h8h50y', '1h8glkz', '1h8g50t', '1h8cue9', '1h8cmw5', '1h8b5hh', '1h8b1ik', '1h8b03i', '1h8aj0a', '1h892yq', '1h88f75', '1h87kt3', '1h87a6u', '1h848tt', '1h829qs', '1h81uv4', '1h813eu', '1h80z8i', '1h7z0oa', '1h7x1in', '1h7vtze', '1h7ups3', '1h7sz7d', '1h7swwh', '1h7qtzb', '1h7pwsc', '1h7o9vp', '1h7naik', '1h7na49', '1h7lbr2', '1h7kz6j', '1h7kb2v', '1h7k5e0', '1h7jhe7', '1h7h4re', '1h7g55b', '1h7f9zv', '1h7eo7c', '1h7dvkr', '1h7aic5', '1h7a4sa', '1h74irl', '1h745jt', '1h71115', '1h6zf4y', '1h6yt0d', '1h6xbny', '1h6vx4p', '1h6vhar', '1h6rslq', '1h6rank', '1h6r9vv', '1h6pm9q', '1h6nfv4', '1h6layg', '1h6k53z', '1h6hqyr', '1gx

In [18]:
remaining_post_ids[0]

'1h3lod0'

In [ ]:
import requests
url ="http://localhost:5678/webhook-test/post_comm"
body = {
    "post_id": '1h9gtbi'
}
resp = requests.post(url, json=body)
print(resp.json())

In [21]:
failed_post_ids = []
import requests
url = "http://localhost:5678/webhook/post_comm"
count = 0
for i in remaining_post_ids:
    if df[df['id'] == i]['num_comments'].values[0] <= 0:
        print(f'{i} has no comments')
        failed_post_ids.append(i)
        continue
    count += 1
    body = {
        "post_id": i,
    }
    try:
        resp = requests.post(url, body, timeout=40)
        print(f'{count}. {resp.text}')
    except requests.exceptions.Timeout:
        print(f'{count}. Request for post_id {i} timed out')
    time.sleep(2)

1. Request for post_id 1gn3k6q timed out
2. Request for post_id 1grbyx4 timed out
3. Request for post_id 1guj8hq timed out
4. Request for post_id 1h5enxa timed out
5. Request for post_id 1h1t6v8 timed out
6. Request for post_id 1gumos0 timed out
7. Request for post_id 1h6iy9o timed out
8. Request for post_id 1gx1oh2 timed out
9. Request for post_id 1h5g3a6 timed out
10. Request for post_id 1h1ujjx timed out
11. Request for post_id 1h4wski timed out


: 

In [5]:
res = comms_db.find()
res = list(res)
print(len(res))

7101


In [6]:
comms_df = pd.DataFrame(res)
comms_df.head()

,_id,comm_id,body,parent_id,ups
0,675d989f5eb32f14e7e00e60,lyvcb3q,"Being considered a startup founder, I went to ...",t3_1gzc5xs,131.0
1,675d989f5eb32f14e7e00e61,lyve1vy,Some things at the top of my head: \n \n1. H...,t3_1gzc5xs,63.0
2,675d989f5eb32f14e7e00e62,lyvll3v,I think the most surprising thing is how hard ...,t3_1gzc5xs,26.0
3,675d989f5eb32f14e7e00e63,lyvpeoe,Startups are fast paced… sometimes. Nobody rea...,t3_1gzc5xs,27.0
4,675d989f5eb32f14e7e00e64,lyvrxn9,That VCs know what they’re talking about. \n\n...,t3_1gzc5xs,22.0


In [7]:
comms_df['parent_id'] = comms_df['parent_id'].apply(lambda x: x.split('_')[1])
comms_df.head()

,_id,comm_id,body,parent_id,ups
0,675d989f5eb32f14e7e00e60,lyvcb3q,"Being considered a startup founder, I went to ...",1gzc5xs,131.0
1,675d989f5eb32f14e7e00e61,lyve1vy,Some things at the top of my head: \n \n1. H...,1gzc5xs,63.0
2,675d989f5eb32f14e7e00e62,lyvll3v,I think the most surprising thing is how hard ...,1gzc5xs,26.0
3,675d989f5eb32f14e7e00e63,lyvpeoe,Startups are fast paced… sometimes. Nobody rea...,1gzc5xs,27.0
4,675d989f5eb32f14e7e00e64,lyvrxn9,That VCs know what they’re talking about. \n\n...,1gzc5xs,22.0


post_id,title,selftext,comms_id,body

In [10]:
final_ds = final_res.find()
for i in range(len(comms_df)):
    final_res.update_one({'comms_id': comms_df['comm_id'][i]}, {'$set': {'ups':comms_df['ups'][i]}}, upsert=True)
    

In [11]:
final_ds = final_res.find()
final_df = pd.DataFrame(final_ds)
final_df.head(20)

,_id,post_id,title,selftext,comms_id,comment,ups
0,675d82933c7218fc233dcf49,1h9gtbi,Do you think this startup is worth continuing?,"1,006 people came to my app and I made 9 sales...",m10p48u,"Every popular thing ever, that's not sold in b...",18.0
1,675d82933c7218fc233dcf4a,1h9gtbi,Do you think this startup is worth continuing?,"1,006 people came to my app and I made 9 sales...",m111ol6,9 sales in 9 minutes or 9 years?,13.0
2,675d82933c7218fc233dcf4b,1h9gtbi,Do you think this startup is worth continuing?,"1,006 people came to my app and I made 9 sales...",m10mfqc,This is far from enough information to decide....,7.0
3,675d82933c7218fc233dcf4c,1h9gtbi,Do you think this startup is worth continuing?,"1,006 people came to my app and I made 9 sales...",m10vjmy,I don’t know. You’d have to look at the total...,2.0
4,675d82933c7218fc233dcf4d,1h9gtbi,Do you think this startup is worth continuing?,"1,006 people came to my app and I made 9 sales...",m10mg6p,I will need to understand what market you are ...,1.0
5,675d82943c7218fc233dcf4e,1h9gtbi,Do you think this startup is worth continuing?,"1,006 people came to my app and I made 9 sales...",m116yli,You’ve clearly got interest and a product peop...,1.0
6,675d82943c7218fc233dcf4f,1h9gtbi,Do you think this startup is worth continuing?,"1,006 people came to my app and I made 9 sales...",m1183k7,"Could be better, could be worse. But you've al...",1.0
7,675d82943c7218fc233dcf50,1h9gtbi,Do you think this startup is worth continuing?,"1,006 people came to my app and I made 9 sales...",m11rkxo,Have you considered changing the price? I wond...,1.0
8,675d82943c7218fc233dcf51,1h9gtbi,Do you think this startup is worth continuing?,"1,006 people came to my app and I made 9 sales...",m13gqac,So I would like to know more about the app and...,1.0
9,675d82943c7218fc233dcf52,1h9gtbi,Do you think this startup is worth continuing?,"1,006 people came to my app and I made 9 sales...",m13toqa,"I’ve been in your shoes, and honestly, getting...",1.0


In [4]:

import requests
url = "http://localhost:5678/webhook/post_comm"
count = 0
for i in post_ids:
    count += 1
    body = {
        "post_id": i,
    }
    try:
        resp = requests.post(url, body, timeout=40)
        print(f'{count}. {resp.text}')
    except requests.exceptions.Timeout:
        print(f'{count}. Request for post_id {i} timed out')
    time.sleep(2)

1. {"comm_id":"lyvcb3q","body":"Being considered a startup founder, I went to meetings with other founders for training and connect and whatnot.\n\nI was bathed into a world of show. Big watches, bragging, ideas that have to be kept secret, ties, looks. I was so out of place with me being your average joe. People were looking down on me.\n\nI listened to their pitch, and it was all trending stuff, nothing behind. I was amazed.\n\nWhen I pitched, I made a point of mentioning the 100 customers I had already. Turned out I was the only one with a real product. Not to mention I had customers and was profitable already.\n\nI then realized 99% of startups are just an attempt at a cash grab to live off it for a while, with the added - but optional - goal of selling at the end because that whole equity trade just isn't viable in the long run.\n\nSo now I listen patiently to their pitch, give my honest opinion, and just don't feel inferior because I am not from the same environment as them - whi

In [4]:
final_df = final_res.find()
final_df = list(final_df)
final_df = pd.DataFrame(final_df)
final_df.head()

,_id,post_id,title,selftext,comms_id,comment,ups
0,675d82933c7218fc233dcf49,1h9gtbi,Do you think this startup is worth continuing?,"1,006 people came to my app and I made 9 sales...",m10p48u,"Every popular thing ever, that's not sold in b...",18.0
1,675d82933c7218fc233dcf4a,1h9gtbi,Do you think this startup is worth continuing?,"1,006 people came to my app and I made 9 sales...",m111ol6,9 sales in 9 minutes or 9 years?,13.0
2,675d82933c7218fc233dcf4b,1h9gtbi,Do you think this startup is worth continuing?,"1,006 people came to my app and I made 9 sales...",m10mfqc,This is far from enough information to decide....,7.0
3,675d82933c7218fc233dcf4c,1h9gtbi,Do you think this startup is worth continuing?,"1,006 people came to my app and I made 9 sales...",m10vjmy,I don’t know. You’d have to look at the total...,2.0
4,675d82933c7218fc233dcf4d,1h9gtbi,Do you think this startup is worth continuing?,"1,006 people came to my app and I made 9 sales...",m10mg6p,I will need to understand what market you are ...,1.0


In [11]:
qu_df= df[df['response'] == 'question']
len(qu_df)

636

In [12]:
qu_ids = qu_df['id'].tolist()
qu_ids

['1h9gtbi',
 '1h9bzig',
 '1h9achm',
 '1h9ac56',
 '1h980yx',
 '1h96sie',
 '1h96sbl',
 '1h96d8c',
 '1h94vta',
 '1h94egx',
 '1h92zk6',
 '1h913zy',
 '1h90nxb',
 '1h8wpup',
 '1h8wajl',
 '1h8u5d3',
 '1h8tnov',
 '1h8tgei',
 '1h8ss1k',
 '1h8r9fp',
 '1h8qq7b',
 '1h8prw9',
 '1h8mm84',
 '1h8kbbb',
 '1h8jv31',
 '1h8iw8c',
 '1h8ivih',
 '1h8h50y',
 '1h8glkz',
 '1h8g50t',
 '1h8cue9',
 '1h8cmw5',
 '1h8b5hh',
 '1h8b1ik',
 '1h8b03i',
 '1h8aj0a',
 '1h88f75',
 '1h87a6u',
 '1h848tt',
 '1h829qs',
 '1h813eu',
 '1h7z0oa',
 '1h7x1in',
 '1h7vtze',
 '1h7sz7d',
 '1h7swwh',
 '1h7qtzb',
 '1h7pwsc',
 '1h7naik',
 '1h7lbr2',
 '1h7kb2v',
 '1h7jhe7',
 '1h7h4re',
 '1h7g55b',
 '1h7eo7c',
 '1h7dvkr',
 '1h7aic5',
 '1h7a4sa',
 '1h74irl',
 '1h745jt',
 '1h71115',
 '1h6zf4y',
 '1h6yt0d',
 '1h6xbny',
 '1h6vx4p',
 '1h6vhar',
 '1h6rslq',
 '1h6layg',
 '1h6k53z',
 '1h6hqyr',
 '1h6dt5g',
 '1h6depo',
 '1h6cqec',
 '1h6bvbz',
 '1h69ogf',
 '1h686e1',
 '1h67m4o',
 '1h67gx6',
 '1h61mpu',
 '1h60lom',
 '1h5zslp',
 '1h5yefu',
 '1h5y72m',
 '1h

In [14]:
for i in range(len(final_df)):
    if final_df['post_id'][i] not in qu_ids:
        final_res.delete_one({'post_id': final_df['post_id'][i]})
        print(f"Deleted {final_df['post_id'][i]}")
len(list(final_res.find()))

Deleted 1h9e0an
Deleted 1h9e0an
Deleted 1h9e0an
Deleted 1h9e0an
Deleted 1h9e0an
Deleted 1h9e0an
Deleted 1h9cua9
Deleted 1h9cua9
Deleted 1h9cua9
Deleted 1h9cua9
Deleted 1h9cua9
Deleted 1h9cua9
Deleted 1h9cua9
Deleted 1h9cua9
Deleted 1h9cua9
Deleted 1h965rt
Deleted 1h965rt
Deleted 1h965rt
Deleted 1h9347e
Deleted 1h9347e
Deleted 1h9347e
Deleted 1h9347e
Deleted 1h9347e
Deleted 1h9347e
Deleted 1h9347e
Deleted 1h9347e
Deleted 1h9347e
Deleted 1h9347e
Deleted 1h91q7g
Deleted 1h91q7g
Deleted 1h91q7g
Deleted 1h91q7g
Deleted 1h91q7g
Deleted 1h91q7g
Deleted 1h91q7g
Deleted 1h8pt5k
Deleted 1h8pt5k
Deleted 1h8pt5k
Deleted 1h8pt5k
Deleted 1h8pt5k
Deleted 1h8iqv1
Deleted 1h892yq
Deleted 1h892yq
Deleted 1h87kt3
Deleted 1h87kt3
Deleted 1h87kt3
Deleted 1h87kt3
Deleted 1h87kt3
Deleted 1h87kt3
Deleted 1h81uv4
Deleted 1h80z8i
Deleted 1h80z8i
Deleted 1h80z8i
Deleted 1h80z8i
Deleted 1h80z8i
Deleted 1h80z8i
Deleted 1h80z8i
Deleted 1h80z8i
Deleted 1h80z8i
Deleted 1h80z8i
Deleted 1h80z8i
Deleted 1h80z8i
Deleted 

4982

In [15]:
final_df = final_res.find()
final_df = list(final_df)
final_df = pd.DataFrame(final_df)
final_qu_df = final_df.drop_duplicates(subset=['post_id'])
len(final_qu_df)

599